In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data1 = pd.read_csv('/content/drive/MyDrive/Labelled_Multiclass_Dataset/aggregated_data.csv')

In [4]:
data1

,start_time,end_time_x,total_forward_packets_window,total_backward_packets_window,total_forward_bytes_window,total_backward_bytes_window,average_packet_size_fwd_window,average_packet_size_bwd_window,flow_duration_window,packet_count_window,...,bwd_iat_mean,bwd_iat_max,bwd_iat_min,bwd_iat_tot,bwd_blk_rate_avg,end_time_y,total_packets,total_bytes,Label,BinLabel
0,2024-08-13 08:00:00,2024-08-13 08:00:05,261,158,28218,11769,64.239935,56.293254,1370937322,126,...,0.0,0,0,0,0.0,2024-08-13 08:00:00,2,114,XMasAttack,1
1,2024-08-13 08:00:56,2024-08-13 08:01:01,42,9,6232,671,134.525210,8.519608,208424228,17,...,0.0,0,0,0,0.0,2024-08-13 08:00:00,2,114,XMasAttack,1
2,2024-08-13 08:00:57,2024-08-13 08:01:02,42,9,6232,671,134.525210,8.519608,208424228,17,...,0.0,0,0,0,0.0,2024-08-13 08:00:00,2,114,XMasAttack,1
3,2024-08-13 08:00:58,2024-08-13 08:01:03,42,9,6232,671,134.525210,8.519608,208424228,17,...,0.0,0,0,0,0.0,2024-08-13 08:00:00,2,114,XMasAttack,1
4,2024-08-13 08:00:59,2024-08-13 08:01:04,42,9,6232,671,134.525210,8.519608,208424228,17,...,0.0,0,0,0,0.0,2024-08-13 08:00:00,2,114,XMasAttack,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23981,2024-08-26 20:15:56,2024-08-26 20:16:01,1,0,121,0,121.000000,0.000000,0,1,...,0.0,0,0,0,0.0,2024-08-26 20:15:00,1,62,Benign,0
23982,2024-08-26 20:15:57,2024-08-26 20:16:02,1,0,121,0,121.000000,0.000000,0,1,...,0.0,0,0,0,0.0,2024-08-26 20:15:00,1,62,Benign,0
23983,2024-08-26 20:15:58,2024-08-26 20:16:03,1,0,121,0,121.000000,0.000000,0,1,...,0.0,0,0,0,0.0,2024-08-26 20:15:00,1,62,Benign,0
23984,2024-08-26 20:15:59,2024-08-26 20:16:04,1,0,121,0,121.000000,0.000000,0,1,...,0.0,0,0,0,0.0,2024-08-26 20:15:00,1,62,Benign,0


In [5]:
# Identifying missing values
missing_val = data1.isna().sum()
print(missing_val.loc[missing_val > 0])

Series([], dtype: int64)


In [6]:
# selecting numeric attributes columns from aggregated_data
num_col = list(data1.select_dtypes(include='number').columns)
print(num_col)

['total_forward_packets_window', 'total_backward_packets_window', 'total_forward_bytes_window', 'total_backward_bytes_window', 'average_packet_size_fwd_window', 'average_packet_size_bwd_window', 'flow_duration_window', 'packet_count_window', 'mean_iat_fwd_window', 'stddev_iat_fwd_window', 'min_iat_fwd_window', 'max_iat_fwd_window', 'mean_iat_bwd_window', 'stddev_iat_bwd_window', 'min_iat_bwd_window', 'max_iat_bwd_window', 'flow_rate_packets_window', 'flow_rate_bytes_window', 'flow_direction_ratio_window', 'byte_direction_ratio_window', 'src_ip_entropy_window', 'dst_ip_entropy_window', 'time_diff_seconds', 'src_port', 'dst_port', 'protocol', 'flow_duration', 'total_forward_packets', 'total_backward_packets', 'total_bytes_forward', 'total_bytes_backward', 'mean_packet_length_forward', 'mean_packet_length_backward', 'packet_size_mean', 'flow_iat_mean', 'down_up_ratio', 'subflow_fwd_pkts', 'subflow_bwd_pkts', 'subflow_fwd_byts', 'subflow_bwd_byts', 'fwd_pkt_len_mean', 'fwd_pkt_len_max', 'f

In [7]:
# Checking for infinity or NaN values and replacing them with the median value of the column
data1.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in num_col:
    if data1[col].isnull().sum() > 0:
        median_value = data1[col].median()
        data1[col] = data1[col].fillna(median_value)

In [8]:
# Identifying missing values
missing_val = data1.isna().sum()
print(missing_val.loc[missing_val > 0])

Series([], dtype: int64)


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
# Binary data point detection  - fog simulation

In [10]:
# Load the scaler and PCA
import joblib
scaler = joblib.load('/content/drive/MyDrive/Labelled_Multiclass_Dataset/binary_models/scaler_binary.pkl')
pca = joblib.load('/content/drive/MyDrive/Labelled_Multiclass_Dataset/binary_models/pca_binary.pkl')
model=joblib.load('/content/drive/MyDrive/Labelled_Multiclass_Dataset/binary_models/feedforward_model_binary.pkl')

In [11]:
# sequential processing of all data points in real time

import time
# Real-time simulation function with latency and overall processing time
def sim_time_based_window(file_path, chunk_size=1000, delay=1):
    latencies = []  # List to store latencies for each chunk
    total_start_time = time.time()  # Start time of the entire process

    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        print("\nProcessing new data chunk...")

        # Start tracking the time for processing this chunk
        start_time_latency = time.time()

        print(chunk.columns)

        # Drop the same non-feature columns from the chunk
        X_chunk = chunk.drop(columns=['Label', 'BinLabel', 'src_ip', 'dst_ip', 'start_time', 'end_time_x', 'end_time_y', 'time_diff', 'time_diff_seconds'])
        y_true = chunk['BinLabel']  # Assuming 'BinLabel' is the target variable

        # Check for NaN values and fill with column means if necessary
        if X_chunk.isna().any().any():
            print("Chunk contains NaN values. Filling with column means.")
            X_chunk = X_chunk.fillna(X_chunk.mean())

        # Scale and apply PCA to the new chunk
        X_scaled_chunk = scaler.transform(X_chunk)  # Scale the chunk
        X_pca_chunk = pca.transform(X_scaled_chunk)  # Apply PCA to the scaled chunk

        # Predict using the trained model
        predictions = model.predict(X_pca_chunk)

        # Simulate the results
        for i, pred in enumerate(predictions):
            label = 'Attack' if pred == 1 else 'Benign'
            print(f"Data point {i+1}: {label}")

        # End tracking the time after classification for this chunk
        end_time_latency = time.time()

        # Calculate latency for this chunk
        chunk_latency = end_time_latency - start_time_latency
        latencies.append(chunk_latency)
        print(f"Latency for this chunk: {chunk_latency:.4f} seconds")

        # Simulate delay for real-time processing
        time.sleep(delay)  # You can adjust/remove delay for actual real-time performance

    # Calculate total processing time
    total_end_time = time.time()
    total_processing_time = total_end_time - total_start_time

    # Calculate average latency
    average_latency = sum(latencies) / len(latencies) if latencies else 0

    print(f"\nTotal processing time: {total_processing_time:.4f} seconds")
    print(f"Average latency per chunk: {average_latency:.4f} seconds")

# Run the simulation
sim_time_based_window('/content/drive/MyDrive/Labelled_Multiclass_Dataset/aggregated_data.csv', chunk_size=1000, delay=2)

Streaming output truncated to the last 5000 lines.
Data point 99: Benign
Data point 100: Benign
Data point 101: Benign
Data point 102: Benign
Data point 103: Benign
Data point 104: Benign
Data point 105: Benign
Data point 106: Benign
Data point 107: Benign
Data point 108: Benign
Data point 109: Benign
Data point 110: Benign
Data point 111: Benign
Data point 112: Benign
Data point 113: Benign
Data point 114: Benign
Data point 115: Benign
Data point 116: Benign
Data point 117: Benign
Data point 118: Benign
Data point 119: Benign
Data point 120: Benign
Data point 121: Benign
Data point 122: Benign
Data point 123: Benign
Data point 124: Benign
Data point 125: Benign
Data point 126: Benign
Data point 127: Benign
Data point 128: Benign
Data point 129: Benign
Data point 130: Benign
Data point 131: Benign
Data point 132: Benign
Data point 133: Benign
Data point 134: Benign
Data point 135: Benign
Data point 136: Benign
Data point 137: Benign
Data point 138: Benign
Data point 139: Benign
Data po

In [12]:
# Real-time simulation function with window_duration
def simulate_time_based_window(file_path, chunk_size, window_duration, delay=1):
    sliding_window = pd.DataFrame()
    all_true_labels = []
    all_predictions = []
    latencies = []

    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        print("\nProcessing new data chunk...")

        # Start tracking the time for processing this chunk
        start_time_latency = time.time()

        sliding_window = pd.concat([sliding_window, chunk])
        sliding_window['end_time_x'] = pd.to_datetime(sliding_window['end_time_x'])

        latest_time = sliding_window['end_time_x'].max()
        window_start_time = latest_time - pd.Timedelta(seconds=window_duration)
        sliding_window = sliding_window[sliding_window['end_time_x'] >= window_start_time]

        # print(f"Data within the sliding window:")
        print(sliding_window)

        # Drop the same non-feature columns from the chunk
        X_chunk = sliding_window.drop(columns=['Label', 'BinLabel', 'src_ip', 'dst_ip', 'start_time', 'end_time_x', 'end_time_y', 'time_diff', 'time_diff_seconds'])
        y_true = sliding_window['BinLabel']  # Assuming 'BinLabel' is the target variable

        # Check for NaN values and fill with column means if necessary
        if X_chunk.isna().any().any():
            print("Chunk contains NaN values. Filling with column means.")
            X_chunk = X_chunk.fillna(X_chunk.mean())

        # Scale and apply PCA to the new chunk
        X_scaled_chunk = scaler.transform(X_chunk)  # Scale the chunk
        X_pca_chunk = pca.transform(X_scaled_chunk)  # Apply PCA to the scaled chunk

        # Predict using the trained model
        predictions = model.predict(X_pca_chunk)

        # End tracking the time after classification
        end_time_latency = time.time()

        # Calculate the processing latency for this chunk
        latency = end_time_latency - start_time_latency
        latencies.append(latency)

        print(f"Processing latency for this chunk: {latency:.4f} seconds")

        # Store predictions and true labels
        all_true_labels.extend(y_true.tolist())
        all_predictions.extend(predictions.tolist())

        # Simulate the results
        for i, pred in enumerate(predictions):
            label = 'Attack' if pred == 1 else 'Benign'
            print(f"Data point {i+1}: {label}")

        time.sleep(delay)  # Simulate delay for real-time processing

    # Calculate average latency
    avg_latency = sum(latencies) / len(latencies)
    print(f"Average processing latency: {avg_latency:.4f} seconds")

    return all_true_labels, all_predictions

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# Run the simulation and get predictions and true labels
true_labels, predictions = simulate_time_based_window('/content/drive/MyDrive/Labelled_Multiclass_Dataset/aggregated_data.csv', chunk_size=1000, window_duration=300, delay=1)

# Convert predictions to a NumPy array if they are in list form
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Convert continuous predictions to binary (assuming threshold of 0.5 for binary classification)
predictions = np.where(predictions >= 0.5, 1, 0)

# Ensure that the confusion matrix includes both classes (0 for benign, 1 for attack)
conf_matrix = confusion_matrix(true_labels, predictions, labels=[0, 1])


# Calculate other metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, labels=[0, 1], zero_division=0)
recall = recall_score(true_labels, predictions, labels=[0, 1], zero_division=0)
f1 = f1_score(true_labels, predictions, labels=[0, 1], zero_division=0)

# Specificity: True Negative Rate = TN / (TN + FP)
tn, fp, fn, tp = conf_matrix.ravel()
specificity = tn / (tn + fp)

# Display results
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Specificity: {specificity}")

Streaming output truncated to the last 5000 lines.
2974  2024-08-13 17:54:59 2024-08-13 17:55:04                            13   
2975  2024-08-13 17:55:00 2024-08-13 17:55:05                            13   
2976  2024-08-13 17:55:56 2024-08-13 17:56:01                            90   
2977  2024-08-13 17:55:57 2024-08-13 17:56:02                            90   
2978  2024-08-13 17:55:58 2024-08-13 17:56:03                            90   
2979  2024-08-13 17:55:59 2024-08-13 17:56:04                            90   
2980  2024-08-13 17:56:00 2024-08-13 17:56:05                            90   
2981  2024-08-13 17:56:56 2024-08-13 17:57:01                            78   
2982  2024-08-13 17:56:57 2024-08-13 17:57:02                            78   
2983  2024-08-13 17:56:58 2024-08-13 17:57:03                            78   
2984  2024-08-13 17:56:59 2024-08-13 17:57:04                            78   
2985  2024-08-13 17:57:00 2024-08-13 17:57:05                            78   
2

In [14]:
# Parallel processing simulation
import os
import psutil
import multiprocessing as mp
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Input
from tensorflow.keras.layers import Dropout

# Function to simulate real-time classification for each chunk
def simulate_real_time_classification(chunk, scaler, pca, model):
    latencies = []  # List to store latencies for each chunk
    total_start_time = time.time()  # Start time of the entire process
    print(f"Processing chunk in process ID: {os.getpid()}")
    cpu_percent = psutil.cpu_percent(interval=1)  # Check CPU utilization
    print(f"CPU Utilization in process {os.getpid()}: {cpu_percent}%")

    cpu_count = psutil.cpu_count()
    print(f"Number of CPUs {os.getpid()}:{cpu_count}")

    memory_info = psutil.virtual_memory()
    print(f"Memory Usage {os.getpid()}:{memory_info.percent}%")

    # Start tracking the time for processing this chunk
    start_time_latency = time.time()
    print(chunk.columns)

    # Perform pre-processing
    chunk = chunk.drop(columns=['Label', 'BinLabel', 'src_ip', 'dst_ip', 'start_time', 'end_time_x', 'end_time_y', 'time_diff', 'time_diff_seconds'])
    X_chunk = chunk.fillna(chunk.mean())

    # Scale and apply PCA
    X_scaled_chunk = scaler.transform(X_chunk)
    X_pca_chunk = pca.transform(X_scaled_chunk)

    if isinstance(X_pca_chunk, pd.DataFrame):
        X_pca_chunk = X_pca_chunk.values
    print(f"Data type of X_pca_chunk: {type(X_pca_chunk)}")

    print(f"Scaled chunk: {X_scaled_chunk[:1]}")  # First few rows after scaling
    print(f"PCA-transformed chunk: {X_pca_chunk[:1]}")  # First few rows after PCA
    print(f"PCA-transformed chunk shape: {X_pca_chunk.shape}")

    if np.any(np.isnan(X_pca_chunk)) or np.any(np.isinf(X_pca_chunk)):
          print("X_pca_chunk contains NaN or Infinite values.")

    # Predict
    start_time = time.time()
    print(f"Input shape for prediction: {X_pca_chunk.shape}")

    try:
      print("About to make predictions...")
      predictions_prob = model.predict(X_pca_chunk)
      print(f"Predictions probabilities shape: {predictions_prob.shape}")
      print(f"Predictions probabilities: {predictions_prob[:1]}")  # Log the first few probabilities
      print(f"Predictions probabilities shape: {predictions_prob.shape}")
      predictions = (predictions_prob > 0.5).astype(int)  # For binary classification
      print(f"Predictions: {predictions[:1]}")
      print(f"Predictions: {predictions}")
    except Exception as e:
      print(f"Error during prediction: {e}")

    latency = time.time() - start_time
    print(f"Latency for this chunk: {latency:.4f} seconds")

    # Print predictions for this chunk
    print("Predictions for this chunk:")
    for i, pred in enumerate(predictions):
        label = 'Attack' if pred == 1 else 'Benign'
        print(f"Data point {i+1}: {label}")

    # End tracking the time after classification for this chunk
    end_time_latency = time.time()

    # Calculate latency for this chunk
    chunk_latency = end_time_latency - start_time_latency
    latencies.append(chunk_latency)
    print(f"Latency for this chunk: {chunk_latency:.4f} seconds")

    # Calculate total processing time
    total_end_time = time.time()
    total_processing_time = total_end_time - total_start_time

    # Calculate average latency
    average_latency = sum(latencies) / len(latencies) if latencies else 0

    print(f"\nTotal processing time: {total_processing_time:.4f} seconds")
    print(f"Average latency per chunk: {average_latency:.4f} seconds")

    # Simulate real-time results
    return predictions



In [15]:
# Function to process the dataset in parallel
def parallel_real_time_simulation(dataset, chunk_size, num_processes, scaler, pca, model):
    # Split the data into chunks for parallel processing
    chunks = [dataset[i:i + chunk_size] for i in range(0, len(dataset), chunk_size)]
    print(f"Number of chunks: {len(chunks)}")
    # Create a pool of workers and use starmap to pass multiple arguments to the worker function
    with mp.Pool(processes=num_processes) as pool:
        results = pool.starmap(simulate_real_time_classification, [(chunk, scaler, pca, model) for chunk in chunks])

    return results



In [ ]:
# Assuming the dataset, scaler, PCA, and trained model are already loaded and defined
aggregated_data = pd.read_csv('/content/drive/MyDrive/Labelled_Multiclass_Dataset/aggregated_data.csv')

scaler = joblib.load('/content/drive/MyDrive/Labelled_Multiclass_Dataset/binary_models/scaler_binary.pkl')
pca = joblib.load('/content/drive/MyDrive/Labelled_Multiclass_Dataset/binary_models/pca_binary.pkl')
model=joblib.load('/content/drive/MyDrive/Labelled_Multiclass_Dataset/binary_models/feedforward_model_binary.pkl')

# Define chunk size and number of parallel processes
chunk_size = 1000  # Number of data points per chunk
num_processes = 2  # Number of parallel processes

start_time = time.time()

# Run the parallel real-time simulation
parallel_results = parallel_real_time_simulation(aggregated_data, chunk_size, num_processes, scaler, pca, model)

# You can now analyze the parallel_results
end_time = time.time()
print(f"Total processing time with parallelization: {end_time - start_time:.4f} seconds")

Number of chunks: 24
Processing chunk in process ID: 20173
Processing chunk in process ID: 20174
CPU Utilization in process 20173: 5.5%
Number of CPUs 20173:2
Memory Usage 20173:16.8%
Index(['start_time', 'end_time_x', 'total_forward_packets_window',
       'total_backward_packets_window', 'total_forward_bytes_window',
       'total_backward_bytes_window', 'average_packet_size_fwd_window',
       'average_packet_size_bwd_window', 'flow_duration_window',
       'packet_count_window', 'mean_iat_fwd_window', 'stddev_iat_fwd_window',
       'min_iat_fwd_window', 'max_iat_fwd_window', 'mean_iat_bwd_window',
       'stddev_iat_bwd_window', 'min_iat_bwd_window', 'max_iat_bwd_window',
       'flow_rate_packets_window', 'flow_rate_bytes_window',
       'flow_direction_ratio_window', 'byte_direction_ratio_window',
       'src_ip_entropy_window', 'dst_ip_entropy_window', 'time_diff',
       'time_diff_seconds', 'src_ip', 'dst_ip', 'src_port', 'dst_port',
       'protocol', 'flow_duration', 'total

KeyboardInterrupt: 

In [ ]:
if model is None:
    print("Model is not loaded properly!")
else:
    model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,605 (37.52 KB)

 Trainable params: 3,201 (12.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,404 (25.02 KB)

NameError: name 'X_pca_chunk' is not defined

In [ ]:
# Multi-Classification - Fog simulation

In [18]:
# Load the scaler and PCA
import joblib
scaler = joblib.load('/content/drive/MyDrive/Labelled_Multiclass_Dataset/multi_class_models/scaler_multi.pkl')
pca = joblib.load('/content/drive/MyDrive/Labelled_Multiclass_Dataset/multi_class_models/pca_multi.pkl')
model=joblib.load('/content/drive/MyDrive/Labelled_Multiclass_Dataset/multi_class_models/fedforward_multi.pkl')

In [19]:
import pickle

# Assuming scaler, pca, and model are already defined objects
try:
    # Test serialization for scaler
    pickle.dumps(scaler)
    print("Scaler object is pickleable.")
except Exception as e:
    print(f"Scaler object is not pickleable: {e}")

try:
    # Test serialization for PCA
    pickle.dumps(pca)
    print("PCA object is pickleable.")
except Exception as e:
    print(f"PCA object is not pickleable: {e}")

try:
    # Test serialization for Feed Forward model
    pickle.dumps(model)
    print("Feed Forward model is pickleable.")
except Exception as e:
    print(f"RandomForest model is not pickleable: {e}")

Scaler object is pickleable.
PCA object is pickleable.
Feed Forward model is pickleable.


In [20]:
# sequential processing of all data points in real time for Multi classification

import time
# Real-time simulation function with latency and overall processing time
def sim_time_based_window(file_path, chunk_size=1000, delay=1):
    latencies = []  # List to store latencies for each chunk
    total_start_time = time.time()  # Start time of the entire process

    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        print("\nProcessing new data chunk...")

        # Start tracking the time for processing this chunk
        start_time_latency = time.time()

        print(chunk.columns)

        # Drop the same non-feature columns from the chunk
        X_chunk = chunk.drop(columns=['Label', 'BinLabel', 'src_ip', 'dst_ip', 'start_time', 'end_time_x', 'end_time_y', 'time_diff', 'time_diff_seconds'])
        y_true = chunk['Label']  # Assuming 'Label' is the target variable


        # Check for NaN values and fill with column means if necessary
        if X_chunk.isna().any().any():
            print("Chunk contains NaN values. Filling with column means.")
            X_chunk = X_chunk.fillna(X_chunk.mean())

        # Scale and apply PCA to the new chunk
        X_scaled_chunk = scaler.transform(X_chunk)  # Scale the chunk
        X_pca_chunk = pca.transform(X_scaled_chunk)  # Apply PCA to the scaled chunk

        # Predict using the trained model
        predictions = model.predict(X_pca_chunk)

        # Simulate the results with multi-class labels
        for i, pred in enumerate(predictions):
            print(f"Data point {i+1}: {pred}")

        # End tracking the time after classification for this chunk
        end_time_latency = time.time()

        # Calculate latency for this chunk
        chunk_latency = end_time_latency - start_time_latency
        latencies.append(chunk_latency)
        print(f"Latency for this chunk: {chunk_latency:.4f} seconds")

        # Simulate delay for real-time processing
        time.sleep(delay)  # You can adjust/remove delay for actual real-time performance

    # Calculate total processing time
    total_end_time = time.time()
    total_processing_time = total_end_time - total_start_time

    # Calculate average latency
    average_latency = sum(latencies) / len(latencies) if latencies else 0

    print(f"\nTotal processing time: {total_processing_time:.4f} seconds")
    print(f"Average latency per chunk: {average_latency:.4f} seconds")

# Run the simulation
sim_time_based_window('/content/drive/MyDrive/Labelled_Multiclass_Dataset/aggregated_data.csv', chunk_size=1000, delay=2)

Streaming output truncated to the last 5000 lines.
Data point 305: [9.9999994e-01 6.2638722e-19 2.0664876e-20 4.0883460e-18 1.9274546e-15]
Data point 306: [9.9999994e-01 7.5469366e-21 2.8917096e-22 1.7006051e-20 8.7019434e-17]
Data point 307: [9.9999994e-01 5.2178532e-19 1.6010299e-20 1.5646963e-17 1.1310058e-16]
Data point 308: [9.9999994e-01 5.2178532e-19 1.6010299e-20 1.5646963e-17 1.1310058e-16]
Data point 309: [9.9999994e-01 5.2178532e-19 1.6010299e-20 1.5646963e-17 1.1310058e-16]
Data point 310: [9.9999994e-01 5.2178532e-19 1.6010299e-20 1.5646963e-17 1.1310058e-16]
Data point 311: [9.9999994e-01 1.7649209e-18 5.2368543e-20 6.8142027e-17 2.7587074e-16]
Data point 312: [9.9999994e-01 2.6581956e-19 1.3024824e-20 5.8972849e-19 2.7738834e-15]
Data point 313: [9.9999994e-01 2.6581956e-19 1.3024824e-20 5.8972849e-19 2.7738834e-15]
Data point 314: [9.9999994e-01 2.6581956e-19 1.3024824e-20 5.8972849e-19 2.7738834e-15]
Data point 315: [9.9999994e-01 2.6581956e-19 1.3024824e-20 5.8972849e

In [ ]:
print("Unique prediction values:", set(predictions))

Unique prediction values: {0, 1}


In [21]:
# Real-time simulation function
def simulate_time_based_window(file_path, chunk_size, window_duration, delay=1):
    sliding_window = pd.DataFrame()
    all_true_labels = []
    all_predictions = []
    latencies = []

    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        print("\nProcessing new data chunk...")

        # Start tracking the time for processing this chunk
        start_time_latency = time.time()

        sliding_window = pd.concat([sliding_window, chunk])
        sliding_window['end_time_x'] = pd.to_datetime(sliding_window['end_time_x'])

        latest_time = sliding_window['end_time_x'].max()
        window_start_time = latest_time - pd.Timedelta(seconds=window_duration)
        sliding_window = sliding_window[sliding_window['end_time_x'] >= window_start_time]

        # print(f"Data within the sliding window:")
        print(sliding_window)

        # Drop the same non-feature columns from the chunk
        X_chunk = sliding_window.drop(columns=['Label', 'BinLabel', 'src_ip', 'dst_ip', 'start_time', 'end_time_x', 'end_time_y', 'time_diff', 'time_diff_seconds'])
        y_true = sliding_window['Label']  # Assuming 'BinLabel' is the target variable

        # Check for NaN values and fill with column means if necessary
        if X_chunk.isna().any().any():
            print("Chunk contains NaN values. Filling with column means.")
            X_chunk = X_chunk.fillna(X_chunk.mean())

        # Scale and apply PCA to the new chunk
        X_scaled_chunk = scaler.transform(X_chunk)  # Scale the chunk
        X_pca_chunk = pca.transform(X_scaled_chunk)  # Apply PCA to the scaled chunk

        # Predict using the trained model
        predictions = model.predict(X_pca_chunk)

        # End tracking the time after classification
        end_time_latency = time.time()

        # Calculate the processing latency for this chunk
        latency = end_time_latency - start_time_latency
        latencies.append(latency)

        print(f"Processing latency for this chunk: {latency:.4f} seconds")

        # Store predictions and true labels
        all_true_labels.extend(y_true.tolist())
        all_predictions.extend(predictions.tolist())

        # Simulate the results
        for i, pred in enumerate(predictions):
            print(f"Data point {i+1}: {pred}")

        time.sleep(delay)  # Simulate delay for real-time processing

    # Calculate average latency
    avg_latency = sum(latencies) / len(latencies)
    print(f"Average processing latency: {avg_latency:.4f} seconds")

    return all_true_labels, all_predictions

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# Run the simulation and get predictions and true labels
true_labels, predictions = simulate_time_based_window('/content/drive/MyDrive/Labelled_Multiclass_Dataset/aggregated_data.csv', chunk_size=2000, window_duration=300, delay=1)

# Flatten the lists if they are lists of lists
true_labels_flat = [item for sublist in true_labels for item in sublist]
predictions_flat = [item for sublist in predictions for item in sublist]

# Now find the unique labels
unique_labels = sorted(set(true_labels_flat) | set(predictions_flat))
print("Unique classes in true_labels and predictions:", unique_labels)

# Generate the confusion matrix with only the present labels
conf_matrix = confusion_matrix(true_labels, predictions, labels=unique_labels)

# Calculate metrics for multi-class classification
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, labels=unique_labels, average='macro', zero_division=0)
recall = recall_score(true_labels, predictions, labels=unique_labels, average='macro', zero_division=0)
f1 = f1_score(true_labels, predictions, labels=unique_labels, average='macro', zero_division=0)

# Display results
# Display results
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Processing new data chunk...
               start_time          end_time_x  total_forward_packets_window  \
1974  2024-08-13 14:34:59 2024-08-13 14:35:04                            42   
1975  2024-08-13 14:35:00 2024-08-13 14:35:05                            42   
1976  2024-08-13 14:35:56 2024-08-13 14:36:01                            68   
1977  2024-08-13 14:35:57 2024-08-13 14:36:02                            68   
1978  2024-08-13 14:35:58 2024-08-13 14:36:03                            68   
1979  2024-08-13 14:35:59 2024-08-13 14:36:04                            68   
1980  2024-08-13 14:36:00 2024-08-13 14:36:05                            68   
1981  2024-08-13 14:36:56 2024-08-13 14:37:01                            18   
1982  2024-08-13 14:36:57 2024-08-13 14:37:02                            18   
1983  2024-08-13 14:36:58 2024-08-13 14:37:03                            18   
1984  2024-08-13 14:36:59 2024-08-13 14:37:04                            18   
1985  2024-08-13 14:37

TypeError: '<' not supported between instances of 'str' and 'float'

In [24]:
print("True labels:", true_labels)
print("Predictions:", predictions)

True labels: ['Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'UDPAttack', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benig

In [ ]:
print("Unique classes in true_labels:", set(true_labels))

Unique classes in true_labels: {'UDPAttack', 'HTTPAttack', 'XMasAttack', 'Benign', 'TCPAttack'}


In [ ]:
# Check unique labels in the true labels and predictions
unique_labels = sorted(set(true_labels) | set(predictions))
print("Unique classes in true_labels and predictions:", unique_labels)

Unique classes in true_labels and predictions: ['Benign', 'HTTPAttack', 'TCPAttack', 'UDPAttack', 'XMasAttack']


In [ ]:
import os
import psutil
import multiprocessing as mp
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier

# Function to simulate real-time classification for each chunk
def simulate_real_time_classification(chunk, scaler, pca, model):
    latencies = []  # List to store latencies for each chunk
    total_start_time = time.time()  # Start time of the entire process
    print(f"Processing chunk in process ID: {os.getpid()}")
    cpu_percent = psutil.cpu_percent(interval=1)  # Check CPU utilization
    print(f"CPU Utilization in process {os.getpid()}: {cpu_percent}%")

    cpu_count = psutil.cpu_count()
    print(f"Number of CPUs {os.getpid()}:{cpu_count}")

    memory_info = psutil.virtual_memory()
    print(f"Memory Usage {os.getpid()}:{memory_info.percent}%")

    # Start tracking the time for processing this chunk
    start_time_latency = time.time()
    print(chunk.columns)

    # Perform pre-processing
    chunk = chunk.drop(columns=['Label', 'BinLabel', 'src_ip', 'dst_ip', 'start_time', 'end_time_x', 'end_time_y', 'time_diff', 'time_diff_seconds'])
    X_chunk = chunk.fillna(chunk.mean())

    # Scale and apply PCA
    X_scaled_chunk = scaler.transform(X_chunk)
    X_pca_chunk = pca.transform(X_scaled_chunk)

    # Predict
    start_time = time.time()
    predictions = model.predict(X_pca_chunk)
    latency = time.time() - start_time
    print(f"Latency for this chunk: {latency:.4f} seconds")

    # Print predictions for this chunk
    print("Predictions for this chunk:")
    for i, pred in enumerate(predictions):
        print(f"Data point {i+1}: {pred}")

    # End tracking the time after classification for this chunk
    end_time_latency = time.time()

    # Calculate latency for this chunk
    chunk_latency = end_time_latency - start_time_latency
    latencies.append(chunk_latency)
    print(f"Latency for this chunk: {chunk_latency:.4f} seconds")

    # Calculate total processing time
    total_end_time = time.time()
    total_processing_time = total_end_time - total_start_time

    # Calculate average latency
    average_latency = sum(latencies) / len(latencies) if latencies else 0

    print(f"\nTotal processing time: {total_processing_time:.4f} seconds")
    print(f"Average latency per chunk: {average_latency:.4f} seconds")

    # Simulate real-time results
    return predictions

# Function to process the dataset in parallel
def parallel_real_time_simulation(dataset, chunk_size, num_processes, scaler, pca, model):
    # Split the data into chunks for parallel processing
    chunks = [dataset[i:i + chunk_size] for i in range(0, len(dataset), chunk_size)]

    # Create a pool of workers and use starmap to pass multiple arguments to the worker function
    with mp.Pool(processes=num_processes) as pool:
        results = pool.starmap(simulate_real_time_classification, [(chunk, scaler, pca, model) for chunk in chunks])

    return results

# Assuming the dataset, scaler, PCA, and trained model are already loaded and defined
aggregated_data = pd.read_csv('/content/drive/MyDrive/Labelled_Multiclass_Dataset/aggregated_data.csv')

# Fit the scaler and PCA on the entire dataset (without the non-feature columns)
X = aggregated_data.drop(columns=['Label', 'BinLabel', 'src_ip', 'dst_ip', 'start_time', 'end_time_x', 'end_time_y', 'time_diff', 'time_diff_seconds'])
scaler = StandardScaler().fit(X)
pca = PCA(n_components=0.95).fit(scaler.transform(X))
y = aggregated_data['Label']  # Assuming 'Label' is the target column
model = DecisionTreeClassifier(max_depth = 54).fit(pca.transform(scaler.transform(X)), y)

# Define chunk size and number of parallel processes
chunk_size = 1000  # Number of data points per chunk
num_processes = 4  # Number of parallel processes

start_time = time.time()

# Run the parallel real-time simulation
parallel_results = parallel_real_time_simulation(aggregated_data, chunk_size, num_processes, scaler, pca, model)

# You can now analyze the parallel_results
end_time = time.time()
print(f"Total processing time with parallelization: {end_time - start_time:.4f} seconds")

Streaming output truncated to the last 5000 lines.

Data point 882: Benign

Data point 741: BenignData point 747: BenignData point 883: Benign

Data point 771: Benign

Data point 742: BenignData point 773: BenignData point 748: BenignData point 772: BenignData point 884: Benign



Data point 749: Benign
Data point 743: BenignData point 774: Benign


Data point 885: BenignData point 750: Benign
Data point 744: BenignData point 775: Benign
Data point 776: HTTPAttack
Data point 751: Benign

Data point 745: Benign
Data point 886: BenignData point 752: Benign
Data point 777: HTTPAttack


Data point 778: HTTPAttackData point 746: Benign
Data point 779: HTTPAttackData point 887: Benign

Data point 753: Benign
Data point 754: BenignData point 747: Benign
Data point 780: HTTPAttack
Data point 888: Benign

Data point 748: BenignData point 781: HTTPAttack


Data point 782: HTTPAttackData point 755: BenignData point 889: BenignData point 749: Benign


Data point 890: Benign
Data point 756: BenignD